In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pretrained VGG16 (no top layer)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Custom layers for binary classification
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    r"C:\Users\gupta\OneDrive\Desktop\computer vision project\myenv\small_eye_dataset",
    target_size=(224,224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_gen = datagen.flow_from_directory(
   r"C:\Users\gupta\OneDrive\Desktop\computer vision project\myenv\small_eye_dataset",
    target_size=(224,224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

# Train
model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save trained model
model.save("eye_state_vgg_small_eye_dataset.h5")
